In [15]:
#Data Collection



#Importing Libraries
import numpy as np
import pandas as pd

import sys
from pandas import DataFrame

import json
from pandas.io.json import json_normalize
import csv


import matplotlib.pyplot as plt
%matplotlib inline 
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

from gensim import corpora
from collections import defaultdict
from pprint import pprint

from gensim.models.word2vec import Word2Vec
from gensim.test.utils import common_texts, get_tmpfile

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import common_texts

import re
from sklearn.feature_extraction.text import CountVectorizer

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

import gensim 
from gensim.models import KeyedVectors
import gensim.downloader as api

from operator import itemgetter, attrgetter

from gensim.models.doc2vec import Doc2Vec

In [16]:
#Preprocessing

allposts = pd.read_csv("allPostData.csv", header=0)

#allposts = allposts.dropna()

# Dropping multiple blank columns
# allposts = allposts.drop(["archivedStatus", 
#                           "created_at", 
#                           "created_at__timezone",
#                           "id", 
#                           "owner__about", 
#                           "owner__account_type",
#                           "owner__badge_count",
#                           "owner__buyer_rating",
#                           "owner__created_at",
#                           "owner__email",
#                           "owner__email_verified",
#                           "owner__expo_token",
#                           "owner__false_report_count",
#                           "owner__first_name",
#                           "owner__full_name",
#                           "owner__id",
#                           "owner__is_student",
#                           "owner__last_modified",
#                           "owner__last_name",
#                           "owner__passwordHash",
#                           "owner__profile_pic",
#                           "owner__provider_rating",
#                           "owner__report_count",
#                           "owner__school__address__components__city",
#                           "owner__school__address__components__country",
#                           "owner__school__address__components__county",
#                           "owner__school__address__components__number",
#                           "owner__school__address__components__state",
#                           "owner__school__address__components__street",
#                           "owner__school__address__components__zip_code",
#                           "owner__school__address__formatted",
#                           "owner__school__created_at",
#                           "owner__school__crest",
#                           "owner__school__domains__001",
#                           "owner__school__domains__002",
#                           "owner__school__id",
#                           "owner__school__last_modified",
#                           "owner__school__location__coordinates__001",
#                           "owner__school__location__coordinates__002",
#                           "owner__school__location__type",
#                           "owner__school__name",
#                           "owner__school__website",
#                           "owner__school_id",
#                           "owner__strikes_count",
#                           "owner__stripe_customer__account_balance",
#                           "owner__stripe_customer__created",
#                           "owner__stripe_customer__currency",
#                           "owner__stripe_customer__default_source",
#                           "owner__stripe_customer__delinquent",
#                           "owner__stripe_customer__description",
#                           "owner__stripe_customer__discount",
#                           "owner__stripe_customer__email",
#                           "owner__stripe_customer__id",
#                           "owner__stripe_customer__invoice_prefix",
#                           "owner__stripe_customer__livemode",
#                           "owner__stripe_customer__object",
#                           "owner__stripe_customer__shipping",
#                           "owner__stripe_customer__sources__has_more",
#                           "owner__stripe_customer__sources__object",
#                           "owner__stripe_customer__sources__total_count",
#                           "owner__stripe_customer__sources__url",
#                           "owner__stripe_customer__subscriptions__has_more",
#                           "owner__stripe_customer__subscriptions__object",
#                           "owner__stripe_customer__subscriptions__total_count",
#                           "owner__stripe_customer__subscriptions__url",
#                           "owner__stripe_customer__tax_info",
#                           "owner__stripe_customer__tax_info_verification",
#                           "owner__subscriptions",
#                           "photos__001",
#                           "photos__002",
#                           "photos__003",
#                           "photos__004"
#                          ], axis=1) 

#allposts.head()

In [17]:
# only select relevant columns

allposts = allposts[['description', 'title']]

In [18]:
allposts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 811 entries, 0 to 810
Data columns (total 2 columns):
description    811 non-null object
title          811 non-null object
dtypes: object(2)
memory usage: 12.8+ KB


In [19]:
allposts.shape

(811, 2)

In [20]:
# drop null columns

allposts = allposts.dropna()

In [21]:
allposts = allposts['title']

In [22]:
allposts

0                      Teach Me How To Cook!
1                      Long Boarding Lessons
2                           Personal Trainer
3                       Final Cut Pro Lesson
4                            Resume Building
5                      Motivational Pep Talk
6                     Resume + Cover Letter 
7                          Paracord Bracelet
8                          Video Production 
9                           Consulting Cases
10                           SAS / SQL Tutor
11                         Video Production 
12                            Guitar Lessons
13                   Photography / Portraits
14                  Help Your Tinder Profile
15                      Custom Love Song 😍😍😍
16                                 Gym Buddy
17                      Personal Cheerleader
18                                       Fan
19                                 Lil Ducky
20                      Snowboarding Lessons
21                 Bed Bath & Beyond Drawers
22        

In [23]:
#Data Cleaning

#(1) Define a function that takes in a list of strings and cleans it up

#remove words in stoplist from texts

stoplist = set(stopwords.words('english'))
texts = [[word for word in document.lower().split() if word not in stoplist]
             for document in allposts]

#why does the order of commands matter so much?? check RS_4

In [24]:
#sanity check

print(stoplist)
print(texts)

{'will', 'just', 'd', 'ourselves', 'than', 'we', "won't", 'yourself', "shan't", 'your', "don't", 'until', 'an', 'over', 'because', 'any', 'll', "weren't", "wouldn't", 'nor', "hasn't", 'on', 'm', 'between', 'itself', 'into', 'a', 'them', 'and', 'hadn', 'hers', 'herself', 'wouldn', 'did', 'y', "that'll", 'shan', "isn't", 'ain', 'both', 'before', 'themselves', 'its', 'won', "should've", 'been', 'with', 'about', 'through', 'it', 'being', 'very', 'while', 'these', 'there', 'that', 'do', 'are', 'was', "you'd", 'under', 'her', 'were', 'aren', 'having', 'i', 'so', 'me', "you're", 'had', 'during', 'further', 'when', 'couldn', 'this', 'yourselves', 'yours', 'my', "didn't", 'all', 'himself', 'to', 'how', 'our', 'here', "you've", "aren't", "she's", 'doing', 'the', "it's", 'be', 'why', 'not', 'she', 'haven', 'then', "you'll", 'which', 'if', 'their', 'what', 'once', 'but', 'does', 'ma', 'below', 'hasn', 'shouldn', "doesn't", 'such', 'o', 'those', 'more', 's', 'no', 'he', 'for', 'same', 'above', 'as'

In [25]:
#remove punctuation from texts

txtre_clean = re.compile('<.*?/+!>')
new_clean = [[re.sub(txtre_clean, ' ', word) for word in document] for document in texts]

In [26]:
print(new_clean)
#Lol not working fml wtf

[['teach', 'cook!'], ['long', 'boarding', 'lessons'], ['personal', 'trainer'], ['final', 'cut', 'pro', 'lesson'], ['resume', 'building'], ['motivational', 'pep', 'talk'], ['resume', '+', 'cover', 'letter'], ['paracord', 'bracelet'], ['video', 'production'], ['consulting', 'cases'], ['sas', '/', 'sql', 'tutor'], ['video', 'production'], ['guitar', 'lessons'], ['photography', '/', 'portraits'], ['help', 'tinder', 'profile'], ['custom', 'love', 'song', '😍😍😍'], ['gym', 'buddy'], ['personal', 'cheerleader'], ['fan'], ['lil', 'ducky'], ['snowboarding', 'lessons'], ['bed', 'bath', '&', 'beyond', 'drawers'], ['math', 'tutor'], ['3', 'head', 'lamp'], ['web', 'development'], ['rice', 'cooking', 'pot'], ['get', 'startup', 'job'], ['tennis', 'racket'], ['alarm', 'clock', '/', 'clock'], ['orgo', 'kit'], ['cs', 'tutor'], ['ethernet', 'cable'], ['social', 'media', 'management'], ['mouse'], ['photography'], ['bed', 'raises'], ['tv', 'screen'], ['homecooked', 'dinner'], ['ig', 'social', 'media', 'marke

In [27]:
# printing a cleaned document (which is a list of lists) without stopwords
cleaned_up_document = texts
pprint(cleaned_up_document)


[['teach', 'cook!'],
 ['long', 'boarding', 'lessons'],
 ['personal', 'trainer'],
 ['final', 'cut', 'pro', 'lesson'],
 ['resume', 'building'],
 ['motivational', 'pep', 'talk'],
 ['resume', '+', 'cover', 'letter'],
 ['paracord', 'bracelet'],
 ['video', 'production'],
 ['consulting', 'cases'],
 ['sas', '/', 'sql', 'tutor'],
 ['video', 'production'],
 ['guitar', 'lessons'],
 ['photography', '/', 'portraits'],
 ['help', 'tinder', 'profile'],
 ['custom', 'love', 'song', '😍😍😍'],
 ['gym', 'buddy'],
 ['personal', 'cheerleader'],
 ['fan'],
 ['lil', 'ducky'],
 ['snowboarding', 'lessons'],
 ['bed', 'bath', '&', 'beyond', 'drawers'],
 ['math', 'tutor'],
 ['3', 'head', 'lamp'],
 ['web', 'development'],
 ['rice', 'cooking', 'pot'],
 ['get', 'startup', 'job'],
 ['tennis', 'racket'],
 ['alarm', 'clock', '/', 'clock'],
 ['orgo', 'kit'],
 ['cs', 'tutor'],
 ['ethernet', 'cable'],
 ['social', 'media', 'management'],
 ['mouse'],
 ['photography'],
 ['bed', 'raises'],
 ['tv', 'screen'],
 ['homecooked', 'dinne

In [30]:
#(2) Vectorize the document 

#define training data and train model (use pre-trained)

#Load Google's pre-trained Word2Vec 
#model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

#word_vectors = api.load("glove-wiki-gigaword-100")

#path = get_tmpfile("word2vec.model")
#model = Word2Vec(common_texts, min_count=1)
#model = Word2Vec(cleaned_up_document, min_count=1)
#model.save("word2vec.model")
#model.wv.save_word2vec_format('model.txt', binary=False)
#print(model)


#sentences - (iterable of list of str) – list of lists of tokens
#size - (int, optional) – Dimensionality of the word vectors
#window - (int, optional) – Maximum distance between the current
#and predicted word within a sentence
#min_count (int, optional) – Ignores all words with total 
#frequency lower than this.
#workers (int, optional) – Use these many worker threads to train the model (faster training with multicore machines



#loading wikipedia data 

corpus = api.load('text8')
model = Word2Vec(corpus)

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:833)>

In [31]:
# summarize the loaded model

info = api.info()
print(model)

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:833)>

In [32]:
# summarize vocabulary (using wikipedia data http://mattmahoney.net/dc/textdata.html)

words = list(model.wv.vocab)
print(len(words))

#model.wv - object essentially contains the mapping between words and embeddings
#model = vectorized wikipedia pre-trained corpus 
#word_vectors = vectorized pangea data

NameError: name 'model' is not defined

In [33]:
#sanity check 
vector = model.wv['computer']

NameError: name 'model' is not defined

In [20]:
print(model.wv.syn0.shape)

(71290, 100)


/Users/angelateng/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


In [22]:
#apply vectorization to test data
word_vectors = Word2Vec(cleaned_up_document)

In [37]:
#sanity check
print(word_vectors.wv['food'])

[-1.7661960e-03 -3.7564298e-03  3.6132296e-03 -7.8713882e-04
 -1.1494862e-03  4.2866031e-03  9.4800553e-04 -8.2211720e-04
 -4.3731686e-04 -2.2767044e-03 -4.3388712e-03  2.2976291e-03
 -1.5703499e-03  2.1553673e-03 -1.6350736e-04 -3.0941514e-03
  3.0058390e-03 -9.2567573e-04 -3.4119396e-03 -1.8779953e-03
  2.2534533e-04 -1.7163180e-03  2.8824711e-03  9.9527661e-04
 -3.2324779e-03  1.9906184e-03  6.9063570e-04 -2.4916832e-03
  1.6286620e-03  8.9554186e-04  4.2835891e-04 -1.0209132e-03
 -2.8127921e-03 -6.3069217e-04 -3.7585304e-03 -3.5237193e-03
  4.4697719e-03 -2.5015946e-03  2.5836739e-03  1.1072917e-03
  2.0340246e-03  2.8907226e-03 -1.0388262e-04  4.9305060e-03
  4.5658201e-03 -2.9984519e-03 -4.5103412e-03 -1.9436800e-05
 -3.5500459e-03 -2.2433300e-03  1.0700946e-03  2.6652897e-03
  4.2388574e-03  2.7071789e-03  2.5362342e-03 -3.7039057e-03
  1.6611129e-03  2.5887289e-03 -1.8866800e-04 -4.6084472e-03
  4.2085596e-03 -1.5079424e-03  3.8575779e-03  1.6668206e-04
  1.4764129e-03 -1.56002

In [39]:
print(word_vectors)

#own_words = list(word_vectors.wv.vocab)
#own_words = word_vectors.wv.vocab
#print(own_words)
#seems like we're missing words here

word_vectors

Word2Vec(vocab=65, size=100, alpha=0.025)


In [54]:
#vectorized_document = np.average(array_words, axis = 0)

#goal: for each list of vectors in word_vectors, get the mean. 
#output: we'll have a vectorized document comprised of a list of vector means


# output is numpy 2D array of vectors
#matrix_wordvec = word_vectors.wv.syn0

#matrix_words = model.wv.syn0
#this is for getting the 2d numpy vector (?)


#vectorized_document = np.mean(matrix_wordvec, axis=1)
#taking the mean of each line --> how do I make sure this is right? 
#basically want the ave of each sentence list


list_of_sentence_values = []
for sentence in cleaned_up_document:
    sentiment = np.zeros(100) 
    for word in sentence:
        if word in word_vectors:
            sentiment += word_vectors.wv[word]
        else:
            sentiment += 0
    ave_sentiment = sentiment/len(sentence)
    list_of_sentence_values.append(ave_sentiment)
print(list_of_sentence_values)

[array([ 7.85167940e-05,  1.22930459e-03,  2.03112140e-03,  3.82843777e-04,
       -9.76995798e-04, -1.03582174e-03, -2.42287852e-03, -9.34915268e-04,
       -1.45093375e-03,  4.22556652e-04,  7.79654874e-05,  8.01959890e-04,
       -1.91390887e-03, -8.36170977e-04, -8.54068436e-04,  1.54399150e-03,
       -1.10414709e-04, -5.74943726e-04, -1.27136722e-04, -1.34338843e-04,
        1.13536627e-03, -2.01644201e-04,  3.51145951e-04, -1.27935200e-04,
       -1.86579966e-03, -2.37635872e-03, -1.66140380e-03,  2.01820349e-03,
        3.37357167e-04, -1.59815419e-03, -1.22300262e-04,  7.21284770e-04,
       -5.91303571e-04,  1.26542617e-03,  1.88500667e-03,  7.50691455e-04,
        2.10376712e-03, -9.32320254e-04,  1.24412240e-03,  3.21362488e-04,
       -1.63569394e-03,  5.66171424e-04,  2.38670371e-04, -8.24387418e-04,
        1.25938747e-03, -1.76835048e-03, -3.39136313e-04, -7.96058739e-04,
        2.12947256e-03, -1.75752270e-03, -3.35832650e-04, -1.22794788e-03,
        7.44381105e-04, 

/Users/angelateng/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).


In [56]:
print(list_of_sentence_values.shape)

AttributeError: 'list' object has no attribute 'shape'

In [28]:
print(matrix_wordvec)
#is this right lols --> this is pangea data

[[ 4.8646485e-03 -1.7543889e-03 -4.6381592e-03 ... -6.4241036e-04
  -4.8561948e-03 -3.9645201e-03]
 [ 4.1644368e-03 -1.9585309e-03 -2.2157053e-03 ... -4.7401041e-03
  -1.2929535e-04  3.6379090e-03]
 [-2.8537861e-03 -4.3110433e-03 -1.2589011e-03 ... -1.1897418e-03
  -3.0102655e-03  2.6101754e-03]
 ...
 [-1.8642008e-03 -5.1572867e-04 -9.1064780e-04 ... -4.0190169e-04
   3.3778048e-03  4.2963913e-03]
 [ 4.9855276e-03 -4.5651877e-03 -4.0224371e-03 ... -3.3701097e-06
   3.1115848e-03  3.8482384e-03]
 [-1.7661960e-03 -3.7564298e-03  3.6132296e-03 ...  1.3315555e-03
   4.6660141e-03  4.6958397e-03]]


In [30]:
print(matrix_words)
#this is the matrix of wiki data
print(matrix_words.shape)

print(matrix_wordvec.shape)

[[-0.21695915  0.83611995 -0.23011576 ... -1.9757508   1.1150299
   0.4177372 ]
 [-0.21828878  0.17030029  1.6943575  ... -1.1931375   2.0199475
   1.9767933 ]
 [ 0.0043923  -0.26377773  1.4674052  ... -1.5280236  -0.8035465
   0.17742975]
 ...
 [-0.06865478 -0.03291812  0.10899784 ...  0.02320731 -0.09516497
  -0.09437346]
 [-0.04168268  0.05916854  0.08344258 ...  0.04588417 -0.09913156
  -0.14454325]
 [-0.02411169 -0.00620783  0.20582728 ...  0.02143597 -0.07361057
  -0.10858753]]
(71290, 100)
(65, 100)


In [34]:
print(vectorized_document)
print(vectorized_document.shape)
#this is the vectorized list of lists of sentences

[ 5.82330940e-05 -1.61764765e-04 -5.09401259e-04 -4.85127130e-05
  4.41794778e-04  3.17410479e-04  3.56226315e-04  3.35171528e-04
  5.73978585e-04  5.34166815e-04  2.19704205e-04 -3.39741964e-05
 -6.92885587e-05  1.37005423e-04  3.23244341e-04 -2.11080751e-05
 -7.62268319e-05  9.70452165e-05 -5.35917061e-05 -8.05726100e-04
 -4.14653223e-05  5.23067545e-04 -2.28453664e-05  7.93074651e-05
 -1.54840644e-04 -8.99490915e-05  1.12038666e-04  2.37348708e-04
  4.72007669e-04  1.38328935e-04 -1.14083567e-04 -5.97643775e-05
 -2.19776499e-04 -1.17043499e-04 -8.03308285e-05  3.56610428e-04
 -3.08854622e-04 -2.43073839e-04 -2.86304828e-04 -2.25797543e-04
  2.87285424e-04  4.86183824e-04  5.70987700e-04 -2.27880460e-04
  1.81780750e-04  5.41372219e-06 -2.16170622e-04  3.66715452e-04
 -1.14724717e-04 -1.02896840e-04  6.15469762e-04 -1.25847335e-04
  9.13555486e-05  5.95654536e-04  1.80302566e-04  1.72450469e-04
  2.30912308e-04 -8.24681265e-05 -5.22681621e-05  4.56561072e-04
 -7.79715250e-04 -2.70508

In [28]:
#(3) Find the most similar cosines using matrix dot product

#input_doc_vec: np.array with the vectorized sentence/word

input_doc_vec = matrix_words

#Docs_from_db_vec: list of np.arrays 
#(or an array of np arrays) of the vectorized sentences/words in the database

docs_from_db_vec = vectorized_document

In [41]:
# ranking similarities
docs_with_ranks = []

for db_doc_vec in docs_from_db_vec:
#for vectorized_sentence in input_doc_vec:
    similarity = np.dot(input_doc_vec, docs_from_db_vec)
    docs_with_ranks.append((similarity, docs_from_db_vec))
#pprint(docs_with_ranks[0])
    
#docs with ranks += --> we're adding 2 values, the similarity query and 
#the corresponding vectorized document ????
#basically want to output the similarity rankings and then the data 

In [42]:
pprint(docs_with_ranks)
#shouldn't the dot product be returning a single number? 
#this line seems wrong

[(array([-7.2760601e-04, -7.8558549e-03, -2.9154588e-03, ...,
        3.5451976e-05,  2.2109351e-04,  9.0466063e-05], dtype=float32),
  array([ 1.1618879e-04, -7.4296311e-04,  4.1184720e-04,  5.0001318e-04,
       -8.5845400e-05,  3.6111849e-04, -3.3712893e-04, -5.2932995e-05,
       -2.1530471e-04, -1.3697769e-04, -2.4863734e-04,  3.5137226e-04,
        4.3463806e-04, -4.9058185e-04,  6.7570194e-04, -8.5597468e-04,
        1.2023843e-04, -5.3617032e-04, -1.3198577e-04, -8.0554921e-05,
        5.3257681e-04,  6.2418869e-05, -2.5719043e-04, -3.1853986e-05,
       -1.7020175e-04,  3.4561360e-04,  2.4831994e-04, -2.0890462e-04,
       -1.7973092e-04, -4.5414787e-04, -2.6509917e-04,  6.1060913e-04,
       -1.9089370e-04, -2.2547751e-05,  7.3083145e-05,  4.1685122e-05,
        8.8244215e-06,  5.0463248e-04, -2.3594385e-04, -5.0495702e-05,
       -4.6109208e-06,  2.7945390e-04,  1.0700180e-04,  2.4564902e-04,
        2.5501003e-04, -9.0510811e-04,  7.3309326e-05,  1.8066610e-04,
        3.78

 (array([-7.2760601e-04, -7.8558549e-03, -2.9154588e-03, ...,
        3.5451976e-05,  2.2109351e-04,  9.0466063e-05], dtype=float32),
  array([ 1.1618879e-04, -7.4296311e-04,  4.1184720e-04,  5.0001318e-04,
       -8.5845400e-05,  3.6111849e-04, -3.3712893e-04, -5.2932995e-05,
       -2.1530471e-04, -1.3697769e-04, -2.4863734e-04,  3.5137226e-04,
        4.3463806e-04, -4.9058185e-04,  6.7570194e-04, -8.5597468e-04,
        1.2023843e-04, -5.3617032e-04, -1.3198577e-04, -8.0554921e-05,
        5.3257681e-04,  6.2418869e-05, -2.5719043e-04, -3.1853986e-05,
       -1.7020175e-04,  3.4561360e-04,  2.4831994e-04, -2.0890462e-04,
       -1.7973092e-04, -4.5414787e-04, -2.6509917e-04,  6.1060913e-04,
       -1.9089370e-04, -2.2547751e-05,  7.3083145e-05,  4.1685122e-05,
        8.8244215e-06,  5.0463248e-04, -2.3594385e-04, -5.0495702e-05,
       -4.6109208e-06,  2.7945390e-04,  1.0700180e-04,  2.4564902e-04,
        2.5501003e-04, -9.0510811e-04,  7.3309326e-05,  1.8066610e-04,
        3.78

In [43]:
pprint(similarity)

#sanity check --> ok, i guess it returns one number per vector pair 
#does it matter that dtype = float? 

array([-7.2760601e-04, -7.8558549e-03, -2.9154588e-03, ...,
        3.5451976e-05,  2.2109351e-04,  9.0466063e-05], dtype=float32)


In [44]:
#docs_with_ranks.tolist()
#sorted(docs_with_ranks)
#docs_with_ranks.sort

#sorted((docs_with_ranks).all())

#sorted(docs_with_ranks, key=similarity)

#sorted(docs_with_ranks, key=attrgetter('similarity'))

#similarity_sort = [item for item in docs_with_ranks.sort()]  

# vectorized_document.similar_by_vector([1,2])

pprint(docs_with_ranks[0])


(array([-7.2760601e-04, -7.8558549e-03, -2.9154588e-03, ...,
        3.5451976e-05,  2.2109351e-04,  9.0466063e-05], dtype=float32),
 array([ 1.1618879e-04, -7.4296311e-04,  4.1184720e-04,  5.0001318e-04,
       -8.5845400e-05,  3.6111849e-04, -3.3712893e-04, -5.2932995e-05,
       -2.1530471e-04, -1.3697769e-04, -2.4863734e-04,  3.5137226e-04,
        4.3463806e-04, -4.9058185e-04,  6.7570194e-04, -8.5597468e-04,
        1.2023843e-04, -5.3617032e-04, -1.3198577e-04, -8.0554921e-05,
        5.3257681e-04,  6.2418869e-05, -2.5719043e-04, -3.1853986e-05,
       -1.7020175e-04,  3.4561360e-04,  2.4831994e-04, -2.0890462e-04,
       -1.7973092e-04, -4.5414787e-04, -2.6509917e-04,  6.1060913e-04,
       -1.9089370e-04, -2.2547751e-05,  7.3083145e-05,  4.1685122e-05,
        8.8244215e-06,  5.0463248e-04, -2.3594385e-04, -5.0495702e-05,
       -4.6109208e-06,  2.7945390e-04,  1.0700180e-04,  2.4564902e-04,
        2.5501003e-04, -9.0510811e-04,  7.3309326e-05,  1.8066610e-04,
        3.7857

In [45]:
np.sort(vectorized_document)

array([-9.0510811e-04, -8.5597468e-04, -7.4296311e-04, -6.4009405e-04,
       -6.1266019e-04, -5.7433994e-04, -5.3617032e-04, -5.1228155e-04,
       -5.0345092e-04, -4.9058185e-04, -4.8094519e-04, -4.5414787e-04,
       -4.5091278e-04, -4.1009716e-04, -4.0001990e-04, -3.5208036e-04,
       -3.3712893e-04, -3.2047502e-04, -3.1924038e-04, -3.1670128e-04,
       -2.8700178e-04, -2.8295862e-04, -2.7502768e-04, -2.7380852e-04,
       -2.7241724e-04, -2.6509917e-04, -2.5841800e-04, -2.5719043e-04,
       -2.4863734e-04, -2.4601928e-04, -2.3594385e-04, -2.3109549e-04,
       -2.1530471e-04, -2.0890462e-04, -1.9119307e-04, -1.9089370e-04,
       -1.7973092e-04, -1.7462514e-04, -1.7020175e-04, -1.6463583e-04,
       -1.6118876e-04, -1.5413226e-04, -1.4530991e-04, -1.4279278e-04,
       -1.3697769e-04, -1.3198577e-04, -1.0501011e-04, -8.5845400e-05,
       -8.0554921e-05, -5.9638951e-05, -5.2932995e-05, -5.0495702e-05,
       -3.1853986e-05, -2.2547751e-05, -1.4225365e-05, -4.6109208e-06,
      

In [46]:
np.any(sorted(docs_with_ranks))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [35]:
np.sort(docs_with_ranks, axis=0)

ValueError: setting an array element with a sequence.

def find_most_similar(input_doc_vec, docs_from_db_vec):
    input_doc_vec = vectorize_document(clean)
    docs_from_db_vec = vectorize_document(clean)
    #would these 2 be the same now? since we only have one dataset that hasn't been split into training/testing?
    docs_with_ranks = []
    for db_doc_vec in docs_from_db_vec: 
        similarity = np.dot(input_doc_vec, db_doc_vec)
        docs_with_ranks += [(similarity, docs_db_vec)] 
    return sort_by_similarity(docs_with_ranks)


def process_new_document(document): 
    ```document: a raw document that the user provides```
        cleaned_document = clean_raw_document(document)
        vectorized_document = vectorize_document(cleaned_document)
        docs_from_db_vec = load_from_db(filepath_to_csv_or_db)
        save(vectorized_document, filepath_to_csv_or_db) <- might be a bit more complicated, since you’ll want to save it into a table along with the other document vectors that your users have input earlier, along with an index of some kind that lets you look the vector up
        sorted_documents = find_most_similar(vectorized_document, docs_from_db_vec)

    return sorted_documents
